In [ ]:
import os, shutil
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.utils import image_dataset_from_directory
from tensorflow import keras
from keras import layers
from keras import models
import tensorflow as tf
from keras.preprocessing import image
tf.config.list_physical_devices('GPU')

In [ ]:
train_dir = '../Generator/dataset/train'
test_dir = '../Generator/dataset/test'

train_square_dir = train_dir + '/square'
train_circle_dir = train_dir + '/circle'
train_none_dir = train_dir + '/none'
train_circle_square_dir = train_dir + '/square_circle'
test_cirlce_square_dir = test_dir + '/square_and_circle'

In [ ]:
IMG_SIZE = 500
train_dataset = image_dataset_from_directory(train_dir, image_size=(IMG_SIZE, IMG_SIZE), batch_size=32)
test_dataset = image_dataset_from_directory(test_dir, image_size=(IMG_SIZE, IMG_SIZE), batch_size=32)

In [ ]:
train_images = np.array([])
train_labels = np.array([])
for data_batch, labels_batch in train_dataset:
    data_batch = np.array(data_batch)
    data_batch.astype('float32') / 255
    train_images = np.append(train_images, data_batch)


test_images = np.array([])
test_labels = np.array([])
for data_batch, labels_batch in test_dataset:
    data_batch = np.array(data_batch)
    data_batch.astype('float32') / 255
    test_images = np.append(test_images, data_batch)

#train_images = train_images.reshape(3000, IMG_SIZE, IMG_SIZE, 3)

In [ ]:
train_images = []
train_labels = []

for data_batch, labels_batch in train_dataset.take(2):
    train_images.extend (data_batch.numpy())
    train_labels.extend (labels_batch.numpy())

train_images = np.array(train_images)
train_labels = np.array(train_labels)

print(train_images.shape)
print(train_labels.shape)


train_images = train_images.reshape(3000, 500, 500, 1)
train_images = train_labels.astype('float32') / 255

train_labels = to_categorical(train_labels)


In [ ]:
for data_batch, _ in train_dataset.take(1):
    for i in range(5):
        plt.imshow(data_batch[i].numpy().astype("uint8"))
        plt.show()    

In [ ]:
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Flatten()(x)
x = layers.Dense(512, activation="relu")(x)
outputs = layers.Dense(4, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4), metrics=['acc'])

In [ ]:
train_labels = np.array([])
train_images = np.array([])
for x, y in train_dataset:
    train_labels = np.append(train_labels, y)
    train_images = np.append(train_images, x)

test_labels = np.array([])
test_images = np.array([])
for x, y in test_dataset:
    test_labels = np.append(test_labels, y)
    test_images = np.append(test_images, x)


train_images = train_images.reshape((3000, 500, 500, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((1000, 500, 500, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
history = model.fit(train_dataset, epochs=5, batch_size=32)

In [ ]:

print('Test:', model.predict(test_dataset))

In [ ]:
# model.save('models/04_CNN_02_CatsAndDogs_01_cnn_from_scratch.h5')

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
img = tf.keras.preprocessing.image.load_img('cats_and_dogs_small/validation/dogs/dog.1010.jpg', target_size=(150, 150), interpolation='bilinear')

In [ ]:
plt.imshow(img)
plt.show()

In [ ]:
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)
print(img_array.shape)
result = model.predict(img_array)
print("Result: ", result.round())